# Caso #3

## Modelo Matemático: Problema de Ruteo de Vehículos con Capacidad, Autonomía, Recarga y Peajes

### **Conjuntos**

- $V$: Conjunto de vehículos.
- $N$: Conjunto de todos los nodos.
- $P \subset N$: Conjunto de depósitos.
- $D \subset N$: Conjunto de municipios (clientes).
- $E \subset N$: Conjunto de estaciones de recarga.
- $T \subset N$: Conjunto de peajes.
- $\text{MustPassThrough} \subset N \times N \times E$: Conjunto de tríos $(i,j,e)$ donde el viaje de $i$ a $j$ debe pasar por la estación $e$.

### **Parámetros**

- $F_t$: Tarifa de flete por km (COP/km).
- $C_m$: Costo de mantenimiento por km (COP/km).
- $\text{consumo}$: Consumo de combustible (galones/km).
- $\text{dist}_{i,j}$: Distancia entre el nodo $i$ y el nodo $j$ (km).
- $Q_v$: Capacidad máxima de carga del vehículo $v$.
- $F_{cap,v}$: Capacidad máxima de combustible (autonomía) del vehículo $v$.
- $\text{demand}_j$: Demanda del municipio $j$.
- $\text{fuel\_price}_i$: Precio del combustible en el nodo $i$.
- $\text{toll\_base\_rate}_t$: Tarifa base del peaje $t$.
- $\text{toll\_rate\_per\_ton}_t$: Tarifa adicional por tonelada en el peaje $t$.
- $\text{weight}_v$: Peso del vehículo $v$ en toneladas.
- $\text{max\_weight}_d$: Peso máximo permitido en el municipio $d$.

### **Variables de Decisión**

- **Variable binaria de flujo:**
  $$x_{v,i,j} \in \{0,1\} \quad \forall v \in V, \forall i,j \in N, i \neq j$$
  1 si el vehículo $v$ viaja del nodo $i$ al nodo $j$; 0 en caso contrario.

- **Fracción de uso del vehículo:**
  $$z_v \in [0,1] \quad \forall v \in V$$
  Fracción del vehículo $v$ utilizado.

- **Nivel de combustible:**
  $$f_{v,i,j} \geq 0 \quad \forall v \in V, \forall i,j \in N, i \neq j$$
  Nivel de combustible disponible al viajar de $i$ a $j$.

- **Cantidad de recarga:**
  $$r_{v,i} \geq 0 \quad \forall v \in V, \forall i \in N$$
  Cantidad de recarga de combustible en el nodo $i$.

- **Variable de posición:**
  $$u_{v,i} \geq 0 \quad \forall v \in V, \forall i \in N$$
  Variable de posición en la ruta (para eliminar subtours).

- **Variable de uso de peaje:**
  $$\text{toll\_used}_{v,t} \in \{0,1\} \quad \forall v \in V, \forall t \in T$$
  1 si el vehículo $v$ usa el peaje $t$; 0 en caso contrario.

### **Función Objetivo**

$$
\begin{align*}
\min \sum_{v \in V} \sum_{\substack{i \in N \\ j \in N \\ i \neq j}} (F_t + C_m) \times \text{dist}_{i,j} \times x_{v,i,j} + \sum_{v \in V} \sum_{i \in N} \text{fuel\_price}_i \times r_{v,i} + \sum_{v \in V} \sum_{t \in T} (\text{toll\_base\_rate}_t + \text{toll\_rate\_per\_ton}_t \times \text{weight}_v) \times \text{toll\_used}_{v,t}
\end{align*}
$$

### **Restricciones**

1. **Visita Única por Cliente:**
   $$\sum_{v \in V} \sum_{\substack{i \in N \\ i \neq j}} x_{v,i,j} = 1 \quad \forall j \in D$$

2. **Conservación de Flujo:**
   $$\sum_{\substack{i \in N \\ i \neq k}} x_{v,i,k} = \sum_{\substack{j \in N \\ j \neq k}} x_{v,k,j} \quad \forall v \in V, \forall k \in N \setminus P$$

3. **Salida desde el Depósito:**
   $$\sum_{\substack{i \in P \\ j \in N \\ i \neq j}} x_{v,i,j} = z_v \quad \forall v \in V$$

4. **Retorno al Depósito:**
   $$\sum_{\substack{i \in N \\ j \in P \\ i \neq j}} x_{v,i,j} = z_v \quad \forall v \in V$$

5. **Capacidad de Carga del Vehículo:**
   $$\sum_{j \in D} \text{demand}_j \times \left( \sum_{\substack{i \in N \\ i \neq j}} x_{v,i,j} \right) \leq Q_v \times z_v \quad \forall v \in V$$

6. **Balance de Combustible entre Nodos:**
   $$f_{v,i,j} \geq f_{v,j,j} - \text{consumo} \times \text{dist}_{i,j} \times x_{v,i,j} \quad \forall v \in V, \forall i,j \in N, i \neq j$$

7. **Máxima Capacidad de Combustible:**
   $$f_{v,i,i} \leq F_{cap,v} \times \sum_{\substack{j \in N \\ j \neq i}} x_{v,j,i} \quad \forall v \in V, \forall i \in N$$

8. **Combustible Inicial (Depósito):**
   $$f_{v,i,i} = F_{cap,v} \times z_v \quad \forall v \in V, \forall i \in P$$

9. **Recarga Solo en Estaciones o Depósito:**
   $$r_{v,i} = 0 \quad \forall v \in V, \forall i \in N \setminus (E \cup P)$$

10. **Límite de Recarga por Flujo:**
    $$r_{v,i} \leq F_{cap,v} \times \sum_{\substack{j \in N \\ j \neq i}} x_{v,j,i} \quad \forall v \in V, \forall i \in E \cup P$$

11. **Continuidad en Estaciones:**
    $$\sum_{\substack{i \in N \\ i \neq e}} x_{v,i,e} = \sum_{\substack{j \in N \\ j \neq e}} x_{v,e,j} \quad \forall v \in V, \forall e \in E$$

12. **Continuidad en Peajes:**
    $$\sum_{\substack{i \in N \\ i \neq t}} x_{v,i,t} = \sum_{\substack{j \in N \\ j \neq t}} x_{v,t,j} \quad \forall v \in V, \forall t \in T$$

13. **Entrada y Salida Única del Depósito:**
    $$\sum_{\substack{j \in N \\ j \neq \text{PTO01}}} x_{v,\text{PTO01},j} \leq 1 \quad \forall v \in V$$

    $$\sum_{\substack{i \in N \\ i \neq \text{PTO01}}} x_{v,i,\text{PTO01}} \leq 1 \quad \forall v \in V$$

14. **Límite de Peso en Municipios:**
    $$\text{weight}_v \times \sum_{\substack{i \in N \\ i \neq d}} x_{v,i,d} \leq \text{max\_weight}_d \quad \forall v \in V, \forall d \in D$$

15. **Activación de Uso de Peajes:**
    $$x_{v,t,j} \leq \text{toll\_used}_{v,t} \quad \forall v \in V, \forall t \in T, \forall j \in N, t \neq j$$

16. **Eliminación de Subtours (MTZ):**
    $$u_{v,i} = 0 \quad \forall v \in V, \forall i \in P$$

    $$u_{v,i} \leq |N| \times \sum_{\substack{j \in N \\ j \neq i}} x_{v,j,i} \quad \forall v \in V, \forall i \in N \setminus P$$

    $$u_{v,i} + 1 \leq u_{v,j} + |N| \times (1 - x_{v,i,j}) \quad \forall v \in V, \forall i,j \in N \setminus P, i \neq j$$

17. **Umbral Mínimo de Combustible para Salir:**
    $$f_{v,i,j} \geq 0.3 \times F_{cap,v} \times x_{v,i,j} \quad \forall v \in V, \forall i,j \in N, i \neq j$$

18. **Forzar Paso por Estaciones Específicas:**
    $$x_{v,i,j} \leq x_{v,i,e} + x_{v,e,j} \quad \forall v \in V, \forall (i,j,e) \in \text{MustPassThrough}$$

19. **Si Visita una Estación, Debe Recargar:**
    $$r_{v,e} \geq 0.1 \times \sum_{\substack{i \in N \\ i \neq e}} x_{v,i,e} \quad \forall v \in V, \forall e \in E$$

### **Cambios y Mejoras Realizadas**

1. **Inclusión de Peajes:**
   - Se agregó el conjunto $T$ de peajes.
   - Se añadieron los parámetros `toll_base_rate` y `toll_rate_per_ton` para el cálculo de costos de peaje.
   - Se incorporó la variable `toll_used` para rastrear el uso de peajes.
   - Se actualizó la función objetivo para incluir los costos de peaje.
   - Se agregaron restricciones de continuidad en peajes y activación de uso.

2. **Restricciones de Peso:**
   - Se añadió el parámetro `weight` para el peso de los vehículos.
   - Se incorporó `max_weight` para limitar el acceso a municipios según peso.
   - Se agregó la restricción de límite de peso en municipios.

3. **Mejoría en la Eliminación de Subtours:**
   - Se formalizaron las restricciones MTZ para eliminación efectiva de subtours.

4. **Condiciones de Recarga:**
   - Se refinó la condición para recarga obligatoria en estaciones visitadas.
   - Se especificó un umbral mínimo de combustible para salir de cada nodo.

5. **Paso Obligatorio por Estaciones:**
   - Se formalizó el conjunto `MustPassThrough` y sus restricciones asociadas.


## Notebook ejecutado

In [17]:
%pip install -q amplpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi" ], # pick from over 20 modules including most commercial and open-source solvers
    license_uuid="09f4ede2-4840-4d72-9d94-308ef2f972f7") # your license UUID (e.g., free ampl.com/ce or ampl.com/courses licenses)

Licensed to AMPL Community Edition License for <mariana.lozano.col@gmail.com>.


In [19]:
!apt-get install -y coinor-cbc
!pip install pyomo
!pip install coinor-opt
!pip install amplpy
!pip install pyomo[solvers]
!pip install pandas
!pip install matplotlib


"apt-get" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement coinor-opt (from versions: none)
ERROR: No matching distribution found for coinor-opt

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lec12\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from amplpy import AMPL

# === 1️⃣ Cargar datos ===

clients_df = pd.read_csv('Datos/clients3.csv')
depots_df = pd.read_csv('Datos/depots3.csv')
vehicles_df = pd.read_csv('Datos/vehicles3.csv')
stations_df = pd.read_csv('Datos/stations3.csv')
tolls_df = pd.read_csv('Datos/tolls3.csv')

# === 2️⃣ Preparar conjuntos ===

D = ['MUN' + str(int(row['LocationID'])).zfill(2) for _, row in clients_df.iterrows()]
P = ['PTO01']
E = ['EST' + str(int(row['EstationID'])).zfill(2) for _, row in stations_df.iterrows()]
V = ['VEH' + str(int(row['VehicleID'])).zfill(2) for _, row in vehicles_df.iterrows()]

# Ya no incluimos los peajes como nodos separados
N = P + D + E

# === 3️⃣ Coordenadas y distancias ===

def haversine(coord1, coord2):
    R = 6371
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

coords = {}

tolls_df['BaseRate'] = tolls_df['BaseRate'].fillna(0)
tolls_df['RatePerTon'] = tolls_df['RatePerTon'].fillna(0)

for _, row in depots_df.iterrows():
    coords['PTO01'] = (row['Latitude'], row['Longitude'])

for _, row in stations_df.iterrows():
    key = 'EST' + str(int(row['EstationID'])).zfill(2)
    coords[key] = (row['Latitude'], row['Longitude'])

for _, row in clients_df.iterrows():
    client_id = 'MUN' + str(int(row['LocationID'])).zfill(2)
    coords[client_id] = (row['Latitude'], row['Longitude'])

# Crear un diccionario para mapear municipios a peajes asociados
mun_to_toll = {}
for _, row in tolls_df.iterrows():
    client_id = int(row['ClientID'])
    mun_id = None
    
    # Encontrar el municipio asociado con este ClientID
    for _, client_row in clients_df.iterrows():
        if client_row['ClientID'] == client_id:
            mun_id = 'MUN' + str(int(client_row['LocationID'])).zfill(2)
            break
    
    if mun_id:
        if mun_id not in mun_to_toll:
            mun_to_toll[mun_id] = []
        mun_to_toll[mun_id].append({
            'base_rate': row['BaseRate'] if pd.notna(row['BaseRate']) else 0,
            'rate_per_ton': row['RatePerTon'] if pd.notna(row['RatePerTon']) else 0
        })

dist = {}
for i in N:
    for j in N:
        if i != j:
            dist[(i, j)] = haversine(coords[i], coords[j])

# === Define la función is_point_near_path ===

def is_point_near_path(point, start, end, threshold_km=5):
    """
    Check if a point is near the path from start to end.
    Uses the point-to-line distance formula.
    """
    if start == point or end == point:
        return False
        
    # Convert to radians for calculations
    lat1, lon1 = map(np.radians, start)
    lat2, lon2 = map(np.radians, end)
    lat_p, lon_p = map(np.radians, point)
    
    # Use a simplified approach for checking if a point is near a path
    # Calculate the distance from point to line segment
    # First, get vectors
    v1 = np.array([np.cos(lat1) * np.cos(lon1), np.cos(lat1) * np.sin(lon1), np.sin(lat1)])
    v2 = np.array([np.cos(lat2) * np.cos(lon2), np.cos(lat2) * np.sin(lon2), np.sin(lat2)])
    p = np.array([np.cos(lat_p) * np.cos(lon_p), np.cos(lat_p) * np.sin(lon_p), np.sin(lat_p)])
    
    # Vector from v1 to v2
    v1v2 = v2 - v1
    # Vector from v1 to p
    v1p = p - v1
    
    # Linear algebra to find closest point on line
    # Project v1p onto v1v2
    v1v2_length_squared = np.sum(v1v2**2)
    
    if v1v2_length_squared < 1e-10:  # v1 and v2 are essentially the same point
        return haversine(start, point) < threshold_km
    
    t = max(0, min(1, np.dot(v1p, v1v2) / v1v2_length_squared))
    projection = v1 + t * v1v2
    
    # Convert projection back to lat/lon
    proj_norm = np.sqrt(np.sum(projection**2))
    projection = projection / proj_norm
    lat_proj = np.arcsin(projection[2])
    lon_proj = np.arctan2(projection[1], projection[0])
    
    # Calculate distance from point to projection
    proj_coords = (np.degrees(lat_proj), np.degrees(lon_proj))
    distance = haversine(point, proj_coords)
    
    # Also check if the point is reasonably within the path segment
    # (not just near the extended line)
    distance_start_to_point = haversine(start, point)
    distance_end_to_point = haversine(end, point)
    distance_start_to_end = haversine(start, end)
    
    # If point is far beyond the segment endpoints, don't consider it near the path
    if distance_start_to_point > distance_start_to_end * 1.5 and distance_end_to_point > distance_start_to_end * 1.5:
        return False
    
    return distance < threshold_km

# === Generar conjuntos separados para estaciones ===

must_pass_stations = []

# Para estaciones
for i in N:
    for j in N:
        if i != j and i not in E and j not in E:
            closest_station = None
            min_distance = float('inf')
            for e in E:
                d_ie = dist.get((i, e), 9999)
                d_ej = dist.get((e, j), 9999)
                if d_ie < 15 or d_ej < 15:  # Estaciones a menos de 15 km
                    if d_ie + d_ej < min_distance:
                        min_distance = d_ie + d_ej
                        closest_station = e
            if closest_station:
                must_pass_stations.append((i, j, closest_station))

print(f"✅ MustPassThrough generado con {len(must_pass_stations)} entradas para estaciones")

# === 4️⃣ Escribir archivo .dat para AMPL ===

with open('vrp_data.dat', 'w') as f:
    f.write('set V := {} ;\n'.format(' '.join(V)))
    f.write('set N := {} ;\n'.format(' '.join(N)))
    f.write('set D := {} ;\n'.format(' '.join(D)))
    f.write('set P := {} ;\n'.format(' '.join(P)))
    f.write('set E := {} ;\n'.format(' '.join(E)))

    f.write('param Q :=\n')
    for _, row in vehicles_df.iterrows():
        vid = 'VEH' + str(int(row['VehicleID'])).zfill(2)
        f.write(f'{vid} {row["Capacity"]}\n')
    f.write(';\n')

    f.write('param F_cap :=\n')
    consumo = 0.75  # O el valor que tengas actualizado
    for _, row in vehicles_df.iterrows():
        vid = 'VEH' + str(int(row['VehicleID'])).zfill(2)
        F_cap = row['Range'] * consumo
        f.write(f'{vid} {F_cap}\n')
    f.write(';\n')

    f.write('param demand :=\n')
    for _, row in clients_df.iterrows():
        cid = 'MUN' + str(int(row['LocationID'])).zfill(2)
        f.write(f'{cid} {row["Demand"]}\n')
    f.write(';\n')

    f.write('param fuel_price :=\n')
    for _, row in stations_df.iterrows():
        eid = 'EST' + str(int(row['EstationID'])).zfill(2)
        f.write(f'{eid} {row["FuelCost"]}\n')
    f.write(f'PTO01 13500.0\n')
    for cid in D:
        f.write(f'{cid} 0.0\n')
    f.write(';\n')

    # Parámetros para peajes asociados a municipios
    f.write('param toll_base_rate :=\n')
    for mun in D:
        base_rate = 0
        if mun in mun_to_toll and mun_to_toll[mun]:
            # Si un municipio tiene múltiples peajes, tomamos el valor máximo
            base_rate = max(toll['base_rate'] for toll in mun_to_toll[mun])
        f.write(f'{mun} {base_rate}\n')
    f.write(';\n')

    f.write('param toll_rate_per_ton :=\n')
    for mun in D:
        rate_per_ton = 0
        if mun in mun_to_toll and mun_to_toll[mun]:
            # Si un municipio tiene múltiples peajes, tomamos el valor máximo
            rate_per_ton = max(toll['rate_per_ton'] for toll in mun_to_toll[mun])
        f.write(f'{mun} {rate_per_ton}\n')
    f.write(';\n')

    # Peso de vehículos (convertido a toneladas)
    f.write('param weight :=\n')
    for _, row in vehicles_df.iterrows():
        vid = 'VEH' + str(int(row['VehicleID'])).zfill(2)
        # Asumiendo que el peso está en kg y lo convertimos a toneladas
        weight_tons = row['Weight'] / 1000 if 'Weight' in vehicles_df.columns else row['VehicleID'] * 5  # Peso estimado si no existe
        f.write(f'{vid} {weight_tons}\n')
    f.write(';\n')

    # Restricciones de peso por municipio
    f.write('param max_weight :=\n')
    for _, row in clients_df.iterrows():
        cid = 'MUN' + str(int(row['LocationID'])).zfill(2)
        # Asumiendo que el peso máximo está en toneladas
        max_weight = row['MaxWeight'] if 'MaxWeight' in clients_df.columns else 40  # Valor por defecto
        f.write(f'{cid} {max_weight}\n')
    f.write(';\n')

    f.write('param dist :=\n')
    for (i, j), value in dist.items():
        f.write(f'{i} {j} {value:.4f}\n')
    f.write(';\n')

    # Escribir el conjunto MustPassThrough para estaciones
    if must_pass_stations:
        f.write('set MustPassThrough :=\n')
        for (i, j, e) in must_pass_stations:
            f.write(f'({i}, {j}, {e})\n')
        f.write(';\n')
    else:
        f.write('set MustPassThrough := ;\n')

    # 🚀 Agregar parámetros escalares
    f.write('\nparam Ft := 5000;\n')
    f.write('param Cm := 700;\n')
    f.write('param consumo := 0.75;\n')

print("✅ Archivo vrp_data.dat creado correctamente!")

# === 5️⃣ Ejecutar AMPL ===

ampl = AMPL()
ampl.read('Datos/vrp_model.mod')
ampl.readData('vrp_data.dat')

ampl.setOption('solver', 'gurobi')
ampl.setOption('gurobi_options', 'TimeLimit=600')  # 600 segundos = 10 minutos
print("🚀 Resolviendo con AMPL + Gurobi (límite de 10 minutos)...")
ampl.solve()

# === 6️⃣ Procesar resultados y generar archivo de verificación ===

import csv

print("✅ Costo total global:", ampl.getObjective('TotalCost').value())

z = ampl.getVariable('z')
x = ampl.getVariable('x')
r = ampl.getVariable('r')
u = ampl.getVariable('u')

# 🚀 Cargar precios de estaciones
station_prices = {f"EST{str(int(row['EstationID'])).zfill(2)}": row['FuelCost'] for _, row in stations_df.iterrows()}

# 🚀 Cargar pesos de vehículos
vehicle_weights = {}
for _, row in vehicles_df.iterrows():
    vid = 'VEH' + str(int(row['VehicleID'])).zfill(2)
    # Asumimos que el peso está en kg y lo convertimos a toneladas
    weight_tons = row['Weight'] / 1000 if 'Weight' in vehicles_df.columns else row['VehicleID'] * 5  # Peso estimado si no existe
    vehicle_weights[vid] = weight_tons

with open('verificacion_caso3_mod.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        'VehicleId','LoadCap','FuelCap','RouteSequence','Municipalities','DemandSatisfied',
        'InitLoad','InitFuel','RefuelStops','RefuelAmounts','MunicipalitiesWithTolls','TollCosts','VehicleWeights',
        'Distance','Time','FuelCost','TollCost','TotalCost'
    ])

    for v in ampl.getSet('V'):
        usage = z[v].value()
        if usage > 0.001:
            print(f"\n🛻 Vehículo {v}: Utilizado al {usage*100:.2f}%")

            # 🔗 Recolectar arcos
            arcs = []
            for i in ampl.getSet('N'):
                for j in ampl.getSet('N'):
                    if i != j and x[v, i, j].value() > 0.01:
                        arcs.append((i, j))

            # 🛣️ Reconstruir ruta secuencial
            route = ['PTO01']
            current = 'PTO01'
            visited = set(route)
            while True:
                next_nodes = [j for i, j in arcs if i == current and j not in visited]
                if next_nodes:
                    next_node = next_nodes[0]
                    route.append(next_node)
                    visited.add(next_node)
                    current = next_node
                else:
                    depot_return = [j for i, j in arcs if i == current and j == 'PTO01']
                    if depot_return and 'PTO01' not in route[1:]:
                        route.append('PTO01')
                    break

            print("🛣️ Ruta completa:", ' -> '.join(route))

            # Municipios visitados y demandas
            muni_visited = [n for n in route if n.startswith('MUN')]
            demand_satisfied = len(muni_visited)
            
            # Detalles de demanda por municipio
            demand_details = []
            for m in N:
                if m in muni_visited:
                    demand_details.append(f"{int(ampl.getParameter('demand')[m])}")
                elif m.startswith('MUN'):
                    demand_details.append("0")
                elif m.startswith('EST') or m.startswith('PTO'):
                    demand_details.append("0")
            
            init_load = sum(ampl.getParameter('demand')[m] for m in muni_visited)
            load_cap = ampl.getParameter('Q')[v]
            fuel_cap = ampl.getParameter('F_cap')[v]

            # Distancia total
            total_distance = sum(
                dist[(route[idx], route[idx+1])]
                for idx in range(len(route)-1)
                if (route[idx], route[idx+1]) in dist
            )

            # Tiempo
            time = (total_distance / 40) * 60  # minutos

            # Recargas
            refuels = []
            refuel_amts = []
            fuel_cost = 0
            for est in E:
                amount = r[v, est].value()
                if amount > 0.01:
                    refuels.append(est)
                    refuel_amts.append(f"{amount:.0f}")
                    fuel_cost += amount * station_prices.get(est, 0)
                    print(f"⛽ {est}: {amount:.2f} gal x {station_prices[est]:.0f} = {amount*station_prices[est]:.0f}")

            print(f"💰 Costo combustible vehículo {v}: {fuel_cost:.0f}")

            # Municipios con peajes
            munis_with_tolls = []
            toll_costs = []
            total_toll_cost = 0
            for mun in muni_visited:
                if mun in mun_to_toll and mun_to_toll[mun]:
                    munis_with_tolls.append(mun)
                    base_rate = max(toll['base_rate'] for toll in mun_to_toll[mun])
                    rate_per_ton = max(toll['rate_per_ton'] for toll in mun_to_toll[mun])
                    weight = vehicle_weights.get(v, 0)
                    toll_cost = base_rate + (rate_per_ton * weight)
                    toll_costs.append(f"{toll_cost:.0f}")
                    total_toll_cost += toll_cost
                    print(f"🛣️ Peaje en {mun}: {base_rate:.0f} + {rate_per_ton:.0f} x {weight:.1f}t = {toll_cost:.0f}")

            print(f"💰 Costo peajes vehículo {v}: {total_toll_cost:.0f}")

            # Costos de distancia del vehículo
            distance_cost = sum(
                (ampl.getParameter('Ft').value() + ampl.getParameter('Cm').value()) * dist[(i,j)] * x[v, i, j].value()
                for i, j in dist.keys() if x[v, i, j].value() > 0.01
            )

            total_cost_vehicle = distance_cost + fuel_cost + total_toll_cost
            print(f"💰 Costo total vehículo {v}: {total_cost_vehicle:.0f}")

            writer.writerow([
                v,
                f"{load_cap:.0f}",
                f"{fuel_cap:.0f}",
                '-'.join(route),
                '-'.join(muni_visited),
                '-'.join(demand_details),
                f"{init_load:.0f}",
                f"{fuel_cap:.0f}",
                len(refuels),
                '-'.join(refuel_amts) if refuel_amts else '0',
                len(munis_with_tolls),
                '-'.join(toll_costs) if toll_costs else '0',
                '-'.join([f"{vehicle_weights.get(v, 0):.0f}" for _ in muni_visited]) if muni_visited else '0',
                f"{total_distance:.1f}",
                f"{time:.1f}",
                f"{fuel_cost:.0f}",
                f"{total_toll_cost:.0f}",
                f"{total_cost_vehicle:.0f}"
            ])

print("✅ Archivo de verificación generado correctamente.")

✅ MustPassThrough generado con 598 entradas para estaciones
✅ Archivo vrp_data.dat creado correctamente!
🚀 Resolviendo con AMPL + Gurobi (límite de 10 minutos)...
Gurobi 12.0.1:   lim:time = 600
Gurobi 12.0.1: infeasible problem
254 simplex iterations
1 branching node

suffix dunbdd OUT;
✅ Costo total global: 0.0
✅ Archivo de verificación generado correctamente.
